# Graph algorithms in Python by __[Artyom Iudin](https://github.com/Tomas542/DSaA)__

<div class="alert alert-block alert-info">
<b>Chapter navigation</b> is broken on github. Download .ipynb to use it.
</div>

[Link to my gitlab](https://gitlab.com/Tomas245/dsaa)

# Chapters
0. [Preparetions](#preps)
      1. [Imports](#imports)
      2. [Graph class](#graph)
1. [Breadth First Search](#bfs)
2. [Depth First Search](#dfs)
3. [Levit](#levit)
4. [Dijkstra](#dijkstra)
5. [Bellman-Ford](#bellman_ford)

# Import <a class="anchor" id="import"></a>

Import of modules we need

In [ ]:
import collections

# Graph class <a class="anchor" id="graph"></a>

This class will be one of the representations of graphs as tuple of edges [u, v, weight]

In [ ]:
class Graph:
    def __init__(self, vertices:int) -> None:
        self.V = vertices
        self.graph = []

    def add_edge(self, u:int, v:int, w:int) -> None:
        self.graph.append((u, v, w))

# Breadth First Search <a class="anchor" id="linear"></a>

This algorithm allows to find all vertices that are connected on undirected unweighted graph

In [ ]:
def bfs(matrix: list[list[str]], start: tuple[str, str]) -> set(tuple[str, str]):
    # our queue of vertices we didn'r visit yet
    q = collections.deque()
    q.append(start)

    # visited set for vertices
    visited = set()
    visited.add(start)

    rows = len(matrix)
    cols = len(matrix[0])

    # sides where we will move on our matrix
    directions = [[1, 0], [-1, 0], [0, 1], [0, -1]]    

    while q:
        # taking vertice from the graph
        row, col = q.popleft()
        for dir_row, dir_col in directions:
            # taking new direction in matrix
            r, c = row + dir_row, col + dir_col

            # checkin it in bounds of matrix, not visited vertice
            if ((r in range(rows)) and (c in range(cols)) and 
                (matrix[r][c] == '1') and ((r, c) not in visited)):
                q.append((r, c))
                visited.add((r, c))

    return visited


# Depth First Search <a class="anchor" id="dfs"></a>

Same as bfs, but we poping left from our queue


In [ ]:
def dfs(matrix: list[list[str]], start: tuple[str, str]) -> set(tuple[str, str]):
    # our queue of vertices we didn'r visit yet
    q = collections.deque()
    q.append(start)

    # visited set for vertices
    visited = set()
    visited.add(start)

    rows = len(matrix)
    cols = len(matrix[0])

    # sides where we will move on our matrix
    directions = [[1, 0], [-1, 0], [0, 1], [0, -1]]    

    while q:
        # taking vertice from the graph
        row, col = q.pop()
        for dir_row, dir_col in directions:
            # taking new direction in matrix
            r, c = row + dir_row, col + dir_col

            # checkin it in bounds of matrix, not visited vertice
            if ((r in range(rows)) and (c in range(cols)) and 
                (matrix[r][c] == '1') and ((r, c) not in visited)):
                q.append((r, c))
                visited.add((r, c))

    return visited


# Levit (Pape-Levit) <a class="anchor" id="levit"></a>

We have 3 structure: m0 - caluclated (maybe not finally) vertices, m1 - queue for calculating, m2 - we don't know path and distance. We removing start from m2 to m1 and going through it's neighbours.

<div class="alert alert-block alert-info">
Levit is vulnerable to negative infinite loops inside grapgh
</div>

Time complexity: O(V * E)

In [ ]:
def levit(gr, start: int = 0) -> None:
    # calculated vertices
    m0 = []

    # vertices in queue for calculation
    m1 = collections.deque()
    
    # uncalculated vertices we have no path yet
    m2 = [x for x in range(gr.V)]
    m1.append(m2.pop(start))
    
    # list of distances to vertices
    dist = [float('inf') for _ in range(gr.V)]
    dist[start] = 0

    # predecesors of vertices
    pred = [-1 for _ in range(gr.V)]
    pred[start] = None

    while m1:
        current = m1.popleft()
        for u, v, w in gr.graph:
            if u == current:
                # we have no distance and predecessor for this vertice
                if v in m2:
                    dist[v] = dist[u] + w
                    m2.remove(v)
                    m1.append(v)
                    pred[v] = u
                
                # we have distance and predecessor for this vertice
                elif v in m1:
                    # we found shorter path
                    if dist[v] >= dist[u] + w:
                        dist[v] = dist[u] + w
                        pred[v] = u
                
                # we calculated it and it's neighbours paths
                elif v in m0:
                    # we found shorter way to this vertice
                    if dist[v] > dist[u] + w:
                        dist[v] = dist[u] + w
                        pred[v] = u
                        m0.remove(v)
                        # we recalculate it's neighbours paths
                        m1.appendleft(v)
        
        m0.append(current)
        

# Dijkstra <a class="anchor" id="dijkstra"></a>

In not negative weight graph we take vertices with shortest path to it (we won't find shorter in this case) and calculating it's neighbour's paths.

<div class="alert alert-block alert-info">
Dijkstra is vulnerable to negative weighted edges in grapgh
</div>

Time complexity: O((V + E) * log V)

In [ ]:
def dijkstra(gr, start: int) -> None:
    # our heap of uncalculated vertices
    map_heap = {}

    for i in range(gr.V):
        # till we don't know distance we use inf
        map_heap[i] = float('inf')

    map_heap.pop(start)
    # list of final distances to vertices
    dist = [float('inf') for _ in range(gr.V)]
    dist[start] = 0

    # predecessors of vertices
    pred = [-1 for _ in range(gr.V)]
    pred[start] = None

    # calulating first neigbours of start vertice
    for u, v, w in gr.graph:
        if u == start:
            map_heap[v] = w
            pred[v] = u

    while map_heap:
        smallest = min(map_heap.values())
        
        # taking vertice with shortest path to it
        temp_key = list(map_heap.keys())[list(map_heap.values()).index(smallest)]
        dist[temp_key] = map_heap.pop(temp_key)

        for u, v, w in gr.graph:
            if ((u == temp_key) and (v in map_heap.keys())):
                # recalculating path to vertice
                if map_heap[v] > dist[temp_key] + w:
                    map_heap[v] = dist[temp_key] + w
                    pred[v] = temp_key


# Bellman-Ford <a class="anchor" id="bellman_ford"></a>

In not negative weight graph we take vertices with shortest path to it (we won't find shorter in this case) and calculating it's neighbour's paths.

<div class="alert alert-block alert-info">
Dijkstra is vulnerable to negative weighted edges in grapgh
</div>

Time complexity: O(V * E)

In [ ]:
def bellman_ford(gr, start: int) -> None:
    # list of distances to vertices
    dist = [float('inf') for _ in range(gr.V)]
    dist[start] = 0
    # predecessors of our vertices
    pred = [-1 for _ in range(gr.V)]
    pred[start] = None

    for _ in range(gr.V - 1):
        for u, v, w in gr.graph:
            # found shorter path
            if ((dist[u] != float('inf')) and (dist[u] + w < dist[v])):
                dist[v] = dist[u] + w
                pred[v] = u
    
    # if we'll find shorter path there is negavite loop
    for u, v, w in gr.graph:
        if ((dist[u] != float('inf')) and (dist[u] + w < dist[v])):
            print("Graph contains negative weight cycle")